# Compare Models

Create object to automated model comparisons

In [1]:
import arithmetic_datasets as ad

from tensorflow.keras.layers import Input, Lambda, Dense, concatenate
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
import random
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import tensorflow as tf
import random
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

Using TensorFlow backend.


### ModelComparator

```
mc = ModelComparator()
mc.add_model(name, model)
mc.add_training_set(name, data)

mc.train_models(epochs, batches, **kwargs)

mc.compare_min(metric)
mc.compare_max(metric)

mc.history_for(model_name, training_set_name)
```

In [2]:
class ModelComparator:
    def __init__(self):
        self.models = {}
        self.training_sets = {}
        self.histories = {}
    
    def add_model(self, name, model):
        self.models[name] = model
    
    def add_training_set(self, name, data):
        self.training_sets[name] = data
        
    def train_models(self, **kwargs):
        for model_name in self.models:
            for tset_name in self.training_sets:
                print(f"MODEL {model_name} TRAINING ON DATASET {tset_name}")
                
                model = self.models[model_name]()
                
                history = model.fit(
                    self.training_sets[tset_name][0], self.training_sets[tset_name][1],
                    **kwargs
                )
                
                self.histories[(model_name, tset_name)] = history
    
    def compare_min(self, metric):
        result = {}
        minimum = None
        min_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amin(history[metric])
            if minimum == None or minimum >= result[hkey]:
                minimum = result[hkey]
                min_name = hkey
        return result, {"name" : min_name, "value":minimum}
            
    
    def compare_max(self, metric):
        result = {}
        maximum = None
        max_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amax(history[metric])
            if maximum == None or maximum >= result[hkey]:
                maximum = result[hkey]
                max_name = hkey
        return result, {"name" : max_name, "value": maximum}
    
    def history_for(self, model_name, training_set_name):
        return self.histories[(model_name, training_set_name)]

### Create Models

In [3]:
def nondense_model():
    # Input layer of 3 neurons 
    inp = Input(shape=(1,3))
    
    #128 layer
    d2_out = Dense(128)(inp)

    #grab first, 2nd half of the 128 layer
    d2_out_p1 = Lambda(lambda x: x[:,:,0:64])(d2_out)
    d2_out_p2 = Lambda(lambda x: x[:,:,64:128])(d2_out)

    #64 layer(s)
    d3_out = Dense(64)(d2_out_p1)
    d4_out = Dense(64)(d2_out_p2)

    #grab output nodes from both 64 layers
    d5_out = concatenate([d3_out, d4_out])
    
    o = Dense(1)(d5_out)
    
    model = Model(inp, o)

    model.compile(
        loss="MeanSquaredError",
        metrics=['accuracy']
    )
    
    return model

def dense_model_5L():
    model_5layer = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1024, input_shape=(1,3)),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dense(256),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(1)
    ])
    
    model_5layer.compile(
        loss="MeanSquaredError",
        metrics=['accuracy'] #Acc not working, in testing
    )

    return model_5layer

def dense_model2():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(2048, input_shape=(1,3)))
    
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Dense(64))
    model.add(tf.keras.layers.Dense(1))
    
    model.compile(
        loss="MeanSquaredError",
        metrics = ["accuracy"]
    )
    
    return model

### Datasets

In [4]:
num_examples = 20000
rstart = 1
rend = 10000

# MULT
mult_setX, mult_setY = ad.gen_data_mult(num_examples, rstart, rend)
mult_setX = mult_setX.reshape(num_examples, 1, 3)

# LOG(MULT)
mult_logX = np.log(mult_setX)
mult_logY = np.log(mult_setY)

### Build ModelComparator

In [5]:
mc = ModelComparator()

mc.add_model("nondense_model", nondense_model)
mc.add_model("dense_model", dense_model_5L)
mc.add_model("dense_model v2", dense_model2)

mc.add_training_set("log normal multiply", (mult_logX, mult_logY))
mc.add_training_set("multiplication", (mult_setX, mult_setY))

### Train Models

In [ ]:
num_epochs = 1000
batch_size = num_examples

mc.train_models(
    batch_size=batch_size,
    epochs=num_epochs
)

MODEL nondense_model TRAINING ON DATASET log normal multiply
Epoch 1/1000
1/1 [==============================] - 0s 969us/step - loss: 314.9909 - accuracy: 0.0000e+00
Epoch 2/1000
1/1 [==============================] - 0s 2ms/step - loss: 167.3112 - accuracy: 0.0000e+00
Epoch 3/1000
1/1 [==============================] - 0s 998us/step - loss: 97.7901 - accuracy: 0.0000e+00
Epoch 4/1000
1/1 [==============================] - 0s 997us/step - loss: 56.8424 - accuracy: 0.0000e+00
Epoch 5/1000
1/1 [==============================] - 0s 2ms/step - loss: 31.9014 - accuracy: 0.0000e+00
Epoch 6/1000
1/1 [==============================] - 0s 998us/step - loss: 17.0085 - accuracy: 0.0000e+00
Epoch 7/1000
1/1 [==============================] - 0s 998us/step - loss: 8.5269 - accuracy: 0.0000e+00
Epoch 8/1000
1/1 [==============================] - 0s 998us/step - loss: 4.0008 - accuracy: 0.0000e+00
Epoch 9/1000
1/1 [==============================] - 0s 2ms/step - loss: 1.7648 - accuracy: 0.0000e+00
E

1/1 [==============================] - 0s 3ms/step - loss: 0.9637 - accuracy: 0.0000e+00
Epoch 80/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.5740 - accuracy: 0.0000e+00
Epoch 81/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.3509 - accuracy: 0.0000e+00
Epoch 82/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.2672 - accuracy: 0.0000e+00
Epoch 83/1000
1/1 [==============================] - 0s 997us/step - loss: 0.2253 - accuracy: 0.0000e+00
Epoch 84/1000
1/1 [==============================] - 0s 997us/step - loss: 0.2265 - accuracy: 0.0000e+00
Epoch 85/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.2511 - accuracy: 0.0000e+00
Epoch 86/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.3254 - accuracy: 0.0000e+00
Epoch 87/1000
1/1 [==============================] - 0s 997us/step - loss: 0.4542 - accuracy: 0.0000e+00
Epoch 88/1000
1/1 [==============================] - 0s 998us/step - loss: 0.7075

1/1 [==============================] - 0s 999us/step - loss: 0.6481 - accuracy: 0.0000e+00
Epoch 158/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6830 - accuracy: 0.0000e+00
Epoch 159/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6558 - accuracy: 0.0000e+00
Epoch 160/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6838 - accuracy: 0.0000e+00
Epoch 161/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6485 - accuracy: 0.0000e+00
Epoch 162/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.6695 - accuracy: 0.0000e+00
Epoch 163/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6315 - accuracy: 0.0000e+00
Epoch 164/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.6516 - accuracy: 0.0000e+00
Epoch 165/1000
1/1 [==============================] - 0s 998us/step - loss: 0.6172 - accuracy: 0.0000e+00
Epoch 166/1000
1/1 [==============================] - 0s 2ms/step - loss: 0

1/1 [==============================] - 0s 2ms/step - loss: 0.5320 - accuracy: 0.0000e+00
Epoch 236/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.5568 - accuracy: 0.0000e+00
Epoch 237/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.5298 - accuracy: 0.0000e+00
Epoch 238/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.5569 - accuracy: 0.0000e+00
Epoch 239/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.5271 - accuracy: 0.0000e+00
Epoch 240/1000
1/1 [==============================] - 0s 997us/step - loss: 0.5527 - accuracy: 0.0000e+00
Epoch 241/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.5266 - accuracy: 0.0000e+00
Epoch 242/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.5538 - accuracy: 0.0000e+00
Epoch 243/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.5236 - accuracy: 0.0000e+00
Epoch 244/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.5

1/1 [==============================] - 0s 2ms/step - loss: 0.4519 - accuracy: 0.0000e+00
Epoch 314/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.4752 - accuracy: 0.0000e+00
Epoch 315/1000
1/1 [==============================] - 0s 983us/step - loss: 0.4502 - accuracy: 0.0000e+00
Epoch 316/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.4716 - accuracy: 0.0000e+00
Epoch 317/1000
1/1 [==============================] - 0s 998us/step - loss: 0.4481 - accuracy: 0.0000e+00
Epoch 318/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.4710 - accuracy: 0.0000e+00
Epoch 319/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.4460 - accuracy: 0.0000e+00
Epoch 320/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.4679 - accuracy: 0.0000e+00
Epoch 321/1000
1/1 [==============================] - 0s 997us/step - loss: 0.4448 - accuracy: 0.0000e+00
Epoch 322/1000
1/1 [==============================] - 0s 2ms/step - loss:

...

1/1 [==============================] - 0s 5ms/step - loss: 7.5120 - accuracy: 0.0000e+00
Epoch 271/1000
1/1 [==============================] - 0s 4ms/step - loss: 4.5000 - accuracy: 0.0000e+00
Epoch 272/1000
1/1 [==============================] - 0s 4ms/step - loss: 7.7024 - accuracy: 0.0000e+00
Epoch 273/1000
1/1 [==============================] - 0s 6ms/step - loss: 4.5924 - accuracy: 0.0000e+00
Epoch 274/1000
1/1 [==============================] - 0s 4ms/step - loss: 8.2905 - accuracy: 0.0000e+00
Epoch 275/1000
1/1 [==============================] - 0s 4ms/step - loss: 4.4254 - accuracy: 0.0000e+00
Epoch 276/1000
1/1 [==============================] - 0s 3ms/step - loss: 16.2826 - accuracy: 0.0000e+00
Epoch 277/1000
1/1 [==============================] - 0s 3ms/step - loss: 5.2027 - accuracy: 0.0000e+00
Epoch 278/1000
1/1 [==============================] - 0s 6ms/step - loss: 166.4165 - accuracy: 0.0000e+00
Epoch 279/1000
1/1 [==============================] - 0s 2ms/step - loss: 81

1/1 [==============================] - 0s 4ms/step - loss: 0.2555 - accuracy: 0.0000e+00
Epoch 349/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.2463 - accuracy: 0.0000e+00
Epoch 350/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.2394 - accuracy: 0.0000e+00
Epoch 351/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.2367 - accuracy: 0.0000e+00
Epoch 352/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.2392 - accuracy: 0.0000e+00
Epoch 353/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.2579 - accuracy: 0.0000e+00
Epoch 354/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.3143 - accuracy: 0.0000e+00
Epoch 355/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.5125 - accuracy: 0.0000e+00
Epoch 356/1000
1/1 [==============================] - 0s 3ms/step - loss: 1.1304 - accuracy: 0.0000e+00
Epoch 357/1000
1/1 [==============================] - 0s 5ms/step - loss: 3.807

1/1 [==============================] - 0s 2ms/step - loss: 2.3081 - accuracy: 0.0000e+00
Epoch 427/1000
1/1 [==============================] - 0s 3ms/step - loss: 1.3576 - accuracy: 0.0000e+00
Epoch 428/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.5419 - accuracy: 0.0000e+00
Epoch 429/1000
1/1 [==============================] - 0s 996us/step - loss: 0.3657 - accuracy: 0.0000e+00
Epoch 430/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.2422 - accuracy: 0.0000e+00
Epoch 431/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.2269 - accuracy: 0.0000e+00
Epoch 432/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.2247 - accuracy: 0.0000e+00
Epoch 433/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.2976 - accuracy: 0.0000e+00
Epoch 434/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.4190 - accuracy: 0.0000e+00
Epoch 435/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.7

1/1 [==============================] - 0s 2ms/step - loss: 20.8354 - accuracy: 0.0000e+00
Epoch 505/1000
1/1 [==============================] - 0s 4ms/step - loss: 5.9974 - accuracy: 0.0000e+00
Epoch 506/1000
1/1 [==============================] - 0s 3ms/step - loss: 10.9054 - accuracy: 0.0000e+00
Epoch 507/1000
1/1 [==============================] - 0s 16ms/step - loss: 4.2292 - accuracy: 0.0000e+00
Epoch 508/1000
1/1 [==============================] - 0s 4ms/step - loss: 6.9172 - accuracy: 0.0000e+00
Epoch 509/1000
1/1 [==============================] - 0s 5ms/step - loss: 3.6493 - accuracy: 0.0000e+00
Epoch 510/1000
1/1 [==============================] - 0s 4ms/step - loss: 6.0477 - accuracy: 0.0000e+00
Epoch 511/1000
1/1 [==============================] - 0s 4ms/step - loss: 3.6420 - accuracy: 0.0000e+00
Epoch 512/1000
1/1 [==============================] - 0s 4ms/step - loss: 6.2677 - accuracy: 0.0000e+00
Epoch 513/1000
1/1 [==============================] - 0s 4ms/step - loss: 3.

1/1 [==============================] - 0s 0s/step - loss: 7.9494 - accuracy: 0.0000e+00
Epoch 583/1000
1/1 [==============================] - 0s 0s/step - loss: 3.9430 - accuracy: 0.0000e+00
Epoch 584/1000
1/1 [==============================] - 0s 0s/step - loss: 7.6869 - accuracy: 0.0000e+00
Epoch 585/1000
1/1 [==============================] - 0s 0s/step - loss: 3.8565 - accuracy: 0.0000e+00
Epoch 586/1000
1/1 [==============================] - 0s 0s/step - loss: 7.4388 - accuracy: 0.0000e+00
Epoch 587/1000
1/1 [==============================] - 0s 16ms/step - loss: 3.7977 - accuracy: 0.0000e+00
Epoch 588/1000
1/1 [==============================] - 0s 0s/step - loss: 7.2156 - accuracy: 0.0000e+00
Epoch 589/1000
1/1 [==============================] - 0s 6ms/step - loss: 3.6715 - accuracy: 0.0000e+00
Epoch 590/1000
1/1 [==============================] - 0s 4ms/step - loss: 6.8331 - accuracy: 0.0000e+00
Epoch 591/1000
1/1 [==============================] - 0s 5ms/step - loss: 3.5485 - a

### compare min loss

In [ ]:
models_min_loss, min_loss = mc.compare_min("loss")

In [ ]:
models_min_loss